In [1]:
import ctypes
import numpy as np

In [2]:
def init_kernel_bias(num_inp_channels, kernel_size, num_kernels,mean=0,std=0.01):
    shape = [num_inp_channels, kernel_size, kernel_size, num_kernels]
    weights = std*np.random.randn(*shape) + mean
    # weights/=np.sqrt(num_inp_channels)
    bias = std*np.random.randn(1,num_kernels) + mean
    return np.asfortranarray(weights.astype(np.float32)), np.asfortranarray(bias.astype(np.float32))

In [3]:
w0,b0=init_kernel_bias(num_inp_channels=32,kernel_size=3,num_kernels=64)

In [4]:
inp=np.arange(130*32*32*32).reshape(130,32,32,32).astype(np.float32)

In [5]:
#inp[batches,row,col,d],w0(d,ksz,ksz,num_ker),b0[1,num_ker],stride[row,col]
padding=0
stride=[1,1]
ipp=inp.transpose(0,3,1,2)  #ipp[batches,d,row,col]
output=[]
ksz=w0.shape[1]
num_ker=w0.shape[3]
if not padding: #take care of padding in backprop too
    padding=(ksz-1)//2  #currently don't give 'even' ksz
out_row,out_col=((ipp.shape[2]-ksz+2*padding)//stride[0]+1),((ipp.shape[3]-ksz+2*padding)//stride[1]+1)
batches,d,row,col=ipp.shape
row+=2*padding
col+=2*padding

In [6]:
%%time
padded=np.zeros((batches,d,row,col),dtype=np.float32)
padded[:,:,padding:-padding,padding:-padding]=ipp

CPU times: user 9.78 ms, sys: 6.67 ms, total: 16.4 ms
Wall time: 15.4 ms


In [7]:
# %%timeit
window=(np.arange(ksz)[:,None]*row+np.arange(ksz)).ravel()+np.arange(d)[:,None]*row*col
slider=(np.arange(out_row*stride[0])[:,None]*row+np.arange(out_col*stride[1]))
ind = window.ravel()+slider[::stride[0],::stride[1]].ravel()[:,None]
# bind= np.arange(batches)[:,None]*d*row*col+ind.ravel()
kern = w0.reshape(-1,num_ker,order='F')
# output=(np.dot(np.take(padded, bind).reshape(-1,d*ksz*ksz), kern)).reshape(batches,out_row,out_col,num_ker)

In [8]:
coled=np.empty((batches,*ind.shape),dtype=np.float32)#,order='F')

In [9]:
padded.shape,coled.shape,ind.shape              #make coled=(131072, 288)

((130, 32, 34, 34), (130, 1024, 288), (1024, 288))

In [10]:
%%time
checker=np.empty((batches,*ind.shape),dtype=np.float32)#,order='F')
for i,img in enumerate(padded):      #img[d,row,col]
    # windows(out_row*out_col, ksz*ksz*d) . kernels(d*ksz*ksz,num_ker)
    checker[i]=img.take(ind)

CPU times: user 64.3 ms, sys: 23.2 ms, total: 87.5 ms
Wall time: 86.2 ms


In [11]:
gtake=ctypes.CDLL("libctake.so")

In [19]:
%%timeit
gtake.take(ctypes.c_void_p(padded.ctypes.data),ctypes.c_void_p(ind.ctypes.data),ctypes.c_void_p(coled.ctypes.data),ctypes.c_int(batches),ctypes.c_int(padded[0].size),ctypes.c_int(ind.size))

22 ms ± 605 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [15]:
for i in range(batches):
    if not (coled[i]==checker[i]).all():
        print(i,end=',')

In [17]:
coled[129]

array([[      0.,       0.,       0., ...,       0., 4228127., 4228159.],
       [      0.,       0.,       0., ..., 4228127., 4228159., 4228191.],
       [      0.,       0.,       0., ..., 4228159., 4228191., 4228223.],
       ...,
       [4258688., 4258720., 4258752., ...,       0.,       0.,       0.],
       [4258720., 4258752., 4258784., ...,       0.,       0.,       0.],
       [4258752., 4258784.,       0., ...,       0.,       0.,       0.]],
      dtype=float32)